In [1]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

In [2]:
# Specify visible cuda device
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from parameters import *
from lib.utils import *
from lib.attacks import *
from lib.keras_utils import *

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import regularizers

/home/chawins/.conda/envs/tsa3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [16]:
x_train, y_train, x_test, y_test = load_dataset_mnist()

y_train_cat = keras.utils.to_categorical(y_train)
y_test_cat = keras.utils.to_categorical(y_test)

In [17]:
model_A = build_cnn_mnist()
model_B = build_cnn_mnist_2()
model_C = build_dnn_mnist(784, 300, 4)
model_D = build_dnn_mnist(784, 1200, 6)

In [18]:
model = model_A

In [19]:
epochs = 2000
batch_size = 128
n_batch = len(x_train) // batch_size
min_loss = np.inf

for epoch in range(epochs):
    ind = np.random.permutation(len(x_train))
    for batch in range(n_batch):
        b_ind = ind[batch * batch_size:(batch + 1) * batch_size]
        x, y, y_ = x_train[b_ind], y_train_cat[b_ind], y_train[b_ind]
        # PGD
        x_t = PGD(model, x, y_, grad_fn=None, norm="inf",
                  n_step=40, step_size=0.01, target=False, 
                  init_rnd=0., proj=None, early_stop=False) 
        
        model.train_on_batch(x_t, y)
    
    score = model.evaluate(x_test, y_test_cat, verbose=0)
    print('epoch: {} - {}'.format(epoch, score))
    val_loss = score[0]
    if val_loss < min_loss:
        min_loss = val_loss
        model.save_weights('./tmp_adv.h5')
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter == 5:
            break
    
model.load_weights('./tmp_adv.h5')
print(model.evaluate(x_train, y_train_cat, verbose=0))
print(model.evaluate(x_test, y_test_cat, verbose=0))
model.save_weights('./tmp/adv_train/weights_mnist.h5')

KeyboardInterrupt: 

In [ ]:
def fn_mnist_cnn_adv(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    # Convolutional Layer and pooling #2
    conv2 = tf.layers.conv2d(
        inputs=conv1,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dropout #1
    drop1 = tf.layers.dropout(
        inputs=pool1, rate=0.25, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    # Dense Layer
    drop1_flat = tf.reshape(drop1, [-1, 12 * 12 * 64])
    dense = tf.layers.dense(inputs=drop1_flat, units=128, activation=tf.nn.relu)
    drop2 = tf.layers.dropout(
        inputs=dense, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))

    # Logits Layer
    logits = tf.layers.dense(inputs=drop2, units=10, name="logits")

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
 
    # Structured hinge loss max{0, 1 - (y_label - y_max)}
    # Not so elegant way to index tensor with another tensor
    indices = tf.range(tf.shape(logits)[0])
    gather_ind = tf.stack([indices, labels], axis=1)
    y_label = tf.gather_nd(logits, gather_ind)
    # Get 2 largest outputs
    y_2max = tf.nn.top_k(logits, 2)[0]
    # Find y_max = max(z[i != y])
    i_max = tf.to_int32(tf.argmax(logits, axis=1))
    y_max = tf.where(tf.equal(labels, i_max), y_2max[:, 1],
                     y_2max[:, 0])
    loss = tf.reduce_sum(tf.maximum(0., 100000. - y_label + y_max))
    
    
    
    # Calculate batch accuracy
    tmp = tf.cast(tf.equal(i_max, labels), dtype=tf.float32)
    accuracy = tf.reduce_mean(tmp, name="accuracy")
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)